In [1]:
import pandas as pd
from tqdm import tqdm
import transformers
import torch
import torch.nn.functional as F
import numpy as np
import random

/home/rtx4090/miniconda3/envs/ADS/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df_main = pd.read_csv('multitude.csv')
df_main.head()

,text,label,multi_label,split,language,length,source
0,Der Ausbruch des Coronavirus hat die Entwicklu...,1,text-davinci-003,test,de,174,MULTITuDE_MassiveSumm_spiegel
1,Alex Azar was officially sworn in as the U.S. ...,1,text-davinci-003,train,en,57,MULTITuDE_MassiveSumm_voanews
2,Європейський союз вимагає зупинити розтрату ко...,1,gpt-3.5-turbo,test,uk,105,MULTITuDE_MassiveSumm_interfax
3,"Yesterday, hundreds of Zambian university stud...",1,text-davinci-003,train,en,254,MULTITuDE_MassiveSumm_voanews
4,"In a narrow and highly watched vote, the US Se...",1,gpt-4,train,en,416,MULTITuDE_MassiveSumm_voanews


In [3]:
from sklearn.metrics import auc,roc_curve
def get_roc_metrics(human_preds, ai_preds):
    # human_preds is the ai-generated probabiities of human-text
    # ai_preds is the ai-generated probabiities of AI-text
    if not human_preds or not ai_preds:
            # Handle empty arrays to avoid the IndexError
            return None    # Rest of your code
    fpr, tpr, _ = roc_curve([0] * len(human_preds) + [1] * len(ai_preds), human_preds + ai_preds, pos_label=1)
    roc_auc = auc(fpr, tpr)
    return fpr.tolist(), tpr.tolist(), float(roc_auc)

In [9]:
def accuracy(human_pred, ai_pred):
    count_correct_human = 0
    count_incorrect_human = 0
    count_correct_ai = 0
    count_incorrect_ai = 0
    for i in human_pred:
        if i[0] < 0.5:
            count_correct_human+=1
        else:
            count_incorrect_human +=1
    for i in ai_pred:
        if i[0] > 0.5:
            count_correct_ai +=1
        else:
            count_incorrect_ai +=1
    return  (count_correct_human/len(human_pred)), (count_incorrect_human)/len(human_pred), (count_correct_ai/len(ai_pred)), (count_incorrect_ai)/len(ai_pred)

Languages available are : 
- English (en)
- Spanish (es)  
- Russian (ru) 
- Dutch (nl)  
- Catalan (ca)   
- Czech (cs)  
- German (de)   
- Chinese (zh) 
- Portuguese (pt)   
- Arabic (ar) 
- Ukrainian (uk)  


testing on the test split of every language

In [10]:

device = "cuda"# example: cuda:0
detector_path_or_id = "TrustSafeAI/RADAR-Vicuna-7B"
detector = transformers.AutoModelForSequenceClassification.from_pretrained(detector_path_or_id)
dtokenizer = transformers.AutoTokenizer.from_pretrained(detector_path_or_id)
detector.eval()
detector.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
 

In [13]:
languages = ['de', 'en', 'es', 'nl', 'pt', 'ru', 'zh', 'ar', 'uk', 'cs', 'ca']
results=[]
for i in languages:
    df = df_main[df_main['language'] == i]
    df_test = df[df['split'] == 'test']
    df_test_human = df_test[df_test['label'] == 0]
    df_test_ai = df_test[df_test['label'] == 1]
    print('started evaluating for language:', i)
    output_probs_human = []
    output_probs_ai = []
   # results = []
    with torch.no_grad():
        for i  in tqdm(df_test_human['text']):
            inputs = dtokenizer(i, padding=True, truncation=True, max_length=512, return_tensors="pt")
            inputs = {k:v.to(device) for k,v in inputs.items()}
            output_prob = F.log_softmax(detector(**inputs).logits,-1)[:,0].exp().tolist()
            output_probs_human.append(output_prob)

        for i  in tqdm(df_test_ai['text']):
            inputs = dtokenizer(i, padding=True, truncation=True, max_length=512, return_tensors="pt")
            inputs = {k:v.to(device) for k,v in inputs.items()}
            output_prob = F.log_softmax(detector(**inputs).logits,-1)[:,0].exp().tolist()
            output_probs_ai.append(output_prob)
    fpr, tpr, roc_auc = get_roc_metrics(output_probs_human, output_probs_ai)
    tnr, fpr, tpr, fnr  = accuracy(output_probs_human, output_probs_ai)
    res  = [i, roc_auc, tpr, fnr, tnr, fpr ]
    results.append(res)    
    print('roc_auc:', roc_auc)

    
 

started evaluating for language: de


100%|██████████| 2393/2393 [00:27<00:00, 87.13it/s] 


roc_auc: 0.6751097664993216
started evaluating for language: en


100%|██████████| 2214/2214 [00:24<00:00, 91.00it/s] 


roc_auc: 0.8852983475683132
started evaluating for language: es


...

roc_auc: 0.5415951632543257
started evaluating for language: cs


100%|██████████| 2389/2389 [00:29<00:00, 81.09it/s]


roc_auc: 0.7005776475512767
started evaluating for language: ca


100%|██████████| 2391/2391 [00:29<00:00, 80.11it/s]

roc_auc: 0.6443147915795343


In [15]:
len(results)

11

In [24]:
count =0
import csv
with open('multi.csv', 'w',newline='\n') as f:
        writer = csv.writer(f)
        for res in results:
            writer.writerow(['language ',"aucroc",'tpr','fnr','tnr','fpr'])
            writer.writerow([languages[count],res[0],res[1],res[2],res[4]])

            count+=1

# Performing Translations

In [ ]:
anguages = ['de', 'es', 'nl', 'ru', 'zh', 'ar', 'uk', 'cs', 'ca']
results  = []
for lang in languages:
    print('started translating',lang)
    tokenizer = transformers.AutoTokenizer.from_pretrained(f"Helsinki-NLP/opus-mt-{lang}-en")
    model = transformers.AutoModelForSeq2SeqLM.from_pretrained(f"Helsinki-NLP/opus-mt-{lang}-en")
    model.eval()
    model.to(torch.device('cuda'))
    df = df_main[df_main['language'] == lang]
    df_test = df[df['split'] == 'test']
    df_test_translated = df_test.copy()
    df_test = df_test.reset_index()
    df_test_translated = df_test_translated.reset_index()
    print('model loaded')
    for j in tqdm(range(len(df_test['text']))):
        text = df_test['text'][j]
        text = str(text)
        text = text[:1024]
        inp = text
        input_ids = tokenizer(inp, return_tensors="pt").input_ids
        if len(input_ids[0]) < 512:
            input_ids = input_ids.to(torch.device('cuda'))
            outputs = model.generate(input_ids=input_ids, num_beams=5, num_return_sequences=1)
            text_tr = tokenizer.batch_decode(outputs, skip_special_tokens=True)
            del input_ids
            del outputs
            df_test_translated.loc[[j], 'text'] = text_tr
        else:
             df_test_translated.loc[[j], 'text'] = 'NaN'
        
    del tokenizer
    del model
    df_test_human = df_test_translated[df_test_translated['label'] == 0]
    df_test_ai = df_test_translated[df_test_translated['label'] == 1]
    print('started evaluating for language:', lang, ' after translating to english')
    output_probs_human = []
    output_probs_ai = []
   # results = []
    with torch.no_grad():
        for i  in tqdm(df_test_human['text']):
            inputs = dtokenizer(i, padding=True, truncation=True, max_length=512, return_tensors="pt")
            inputs = {k:v.to(device) for k,v in inputs.items()}
            output_prob = F.log_softmax(detector(**inputs).logits,-1)[:,0].exp().tolist()
            output_probs_human.append(output_prob)

        for i  in tqdm(df_test_ai['text']):
            inputs = dtokenizer(i, padding=True, truncation=True, max_length=512, return_tensors="pt")
            inputs = {k:v.to(device) for k,v in inputs.items()}
            output_prob = F.log_softmax(detector(**inputs).logits,-1)[:,0].exp().tolist()
            output_probs_ai.append(output_prob)
    fpr, tpr, roc_auc = get_roc_metrics(output_probs_human, output_probs_ai)
    tnr, fpr, tpr, fnr  = accuracy(output_probs_human, output_probs_ai)
    res  = [lang, roc_auc, tpr, fnr, tnr, fpr ]
    results.append(res)    
    print('roc_auc:', roc_auc)

    with open('multi_tr.csv', 'a+',newline='\n') as f:
            writer = csv.writer(f)
            writer.writerow([res[0],res[1],res[2],res[3],res[4],res[5]])



Translating portugeese with mbart50-m2en

In [ ]:
from easynmt import EasyNMT
model = EasyNMT('mbart50_m2en')


In [ ]:
df = df_main[df_main['language'] == 'pt']
df_test = df[df['split'] == 'test']
df_test_translated = df_test.copy()
df_test = df_test.reset_index()
df_test_translated = df_test_translated.reset_index()

In [ ]:
for j in tqdm(range(len(df_test['text']))):
        text = df_test['text'][j]
        text = str(text)
        text = text[:1024]
        text_tr = model.translate(text, target_lang='en',source_lang='pt')
        df_test_translated.loc[[j], 'text'] = text_tr

In [ ]:
df_test_human = df_test_translated[df_test_translated['label'] == 0]
df_test_ai = df_test_translated[df_test_translated['label'] == 1]
print('started evaluating for portuguese after translating to english')
output_probs_human = []
output_probs_ai = []
   # results = []
with torch.no_grad():
        for i  in tqdm(df_test_human['text']):
            inputs = dtokenizer(i, padding=True, truncation=True, max_length=512, return_tensors="pt")
            inputs = {k:v.to(device) for k,v in inputs.items()}
            output_prob = F.log_softmax(detector(**inputs).logits,-1)[:,0].exp().tolist()
            output_probs_human.append(output_prob)

        for i  in tqdm(df_test_ai['text']):
            inputs = dtokenizer(i, padding=True, truncation=True, max_length=512, return_tensors="pt")
            inputs = {k:v.to(device) for k,v in inputs.items()}
            output_prob = F.log_softmax(detector(**inputs).logits,-1)[:,0].exp().tolist()
            output_probs_ai.append(output_prob)
fpr, tpr, roc_auc = get_roc_metrics(output_probs_human, output_probs_ai)
tnr, fpr, tpr, fnr  = accuracy(output_probs_human, output_probs_ai)
res  = ['pt', roc_auc, tpr, fnr, tnr, fpr ]

In [ ]:
with open('multi_tr.csv', 'a+',newline='\n') as f:
            writer = csv.writer(f)
            writer.writerow([res[0],res[1],res[2],res[3],res[4],res[5]])